In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'voteherder.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from counts.models import *

# Modelling Election Transfers between Parties on a per constituency basis

Starting with Belfast East in 2017

In [2]:
ballot = Ballot.objects.get(id='nia.belfast-east.2017-03-02')

In [3]:
stages = Stage.objects.filter(ballot__id='nia.belfast-east.2017-03-02')
stages

<QuerySet [<Stage: nia.belfast-east.2017-03-02-1 @ admin>, <Stage: nia.belfast-east.2017-03-02-2 @ admin>, <Stage: nia.belfast-east.2017-03-02-3 @ admin>, <Stage: nia.belfast-east.2017-03-02-4 @ admin>, <Stage: nia.belfast-east.2017-03-02-5 @ admin>, <Stage: nia.belfast-east.2017-03-02-6 @ admin>, <Stage: nia.belfast-east.2017-03-02-7 @ admin>, <Stage: nia.belfast-east.2017-03-02-8 @ admin>, <Stage: nia.belfast-east.2017-03-02-9 @ admin>, <Stage: nia.belfast-east.2017-03-02-10 @ admin>, <Stage: nia.belfast-east.2017-03-02-11 @ admin>]>

In [4]:
stagecells = StageCell.objects.filter(stage__ballot__id='nia.belfast-east.2017-03-02').select_related('stage')

In [5]:
stagecells.first()

<StageCell: 5275.0 for Andy Allen (Ulster Unionist Party) in nia.belfast-east.2017-03-02-1 @ admin>

In [ ]:
party_transfers/party_transfers.sum()*(883/883)

In [6]:
from django.forms.models import model_to_dict
model_to_dict(stagecells.first())

{'stage': UUID('1ecc3ee6-6a11-6f03-a275-00e04c1a5742'),
 'candidate': 8116,
 'count': 5275.0}

In [7]:
import pandas as pd 
df = pd.DataFrame(map(model_to_dict,stagecells))
df['stage'].value_counts()

1ecc3ee6-6a11-6f03-a275-00e04c1a5742    14
1ecc3ee6-6b0c-6f5d-ac5b-00e04c1a5742    14
1ecc3ee6-6c20-6d6a-a314-00e04c1a5742    14
1ecc3ee6-6d28-6eaa-82be-00e04c1a5742    14
1ecc3ee6-6e17-668c-955a-00e04c1a5742    14
1ecc3ee6-6f09-6c79-9ae0-00e04c1a5742    14
1ecc3ee6-701f-6c56-b7ab-00e04c1a5742    14
1ecc3ee6-711b-69f3-93d3-00e04c1a5742    14
1ecc3ee6-7202-6e96-b215-00e04c1a5742    14
1ecc3ee6-72e4-67e5-9915-00e04c1a5742    14
1ecc3ee6-73df-63f6-b791-00e04c1a5742    14
Name: stage, dtype: int64

In [25]:
def renormalize_stagecell(s):
    return dict(
        candidate_id=s.candidate.id,
        candidate_name=s.candidate.name,
        candidate_party_name=s.candidate.party_name,
        candidate_party_id=s.candidate.party_id,
        constituency=s.stage.ballot.constituency,
        count_stage=s.stage.count_stage,
        count=s.count,
        quota=s.stage.ballot.quota
    )
renormalize_stagecell(stagecells.first())

{'candidate_id': 8116,
 'candidate_name': 'Andy Allen',
 'candidate_party_name': 'Ulster Unionist Party',
 'candidate_party_id': 'PP83',
 'constituency': 'belfast-east',
 'count_stage': 1,
 'count': 5275.0,
 'quota': 6727}

In [26]:
df = pd.DataFrame(map(renormalize_stagecell, stagecells.all()))

In [32]:
sdf = df[df['count_stage']==1]
sdf

,candidate_id,candidate_name,candidate_party_name,candidate_party_id,constituency,count_stage,count,quota
0,8116,Andy Allen,Ulster Unionist Party,PP83,belfast-east,1,5275.0,6727
1,19719,Sheila Bodel,Conservative and Unionist Party,PP51,belfast-east,1,275.0,6727
2,13032,Joanne Bunting,Democratic Unionist Party - D.U.P.,PP70,belfast-east,1,6007.0,6727
3,19687,David Douglas,Democratic Unionist Party - D.U.P.,PP70,belfast-east,1,4431.0,6727
4,19745,Séamas de Faoite,SDLP (Social Democratic & Labour Party),PP55,belfast-east,1,250.0,6727
5,13035,Andrew Girvin,Traditional Unionist Voice - TUV,PP680,belfast-east,1,917.0,6727
6,13037,John Kyle,Progressive Unionist Party of Northern Ireland,PP101,belfast-east,1,2658.0,6727
7,2454,Naomi Long,Alliance - Alliance Party of Northern Ireland,PP103,belfast-east,1,7610.0,6727
8,13038,Chris Lyttle,Alliance - Alliance Party of Northern Ireland,PP103,belfast-east,1,5059.0,6727
9,19746,Jordy McKeag,Independent,ynmp-party:2,belfast-east,1,84.0,6727


In [30]:
quota = 6727
# Case where there is only one clear winner; in this case Naomi Long
sdf[sdf['count']>quota]

,candidate_id,candidate_name,candidate_party_name,candidate_party_id,constituency,count_stage,count,quota
7,2454,Naomi Long,Alliance - Alliance Party of Northern Ireland,PP103,belfast-east,1,7610.0,6727


In [41]:
elected = sdf[sdf['count']>quota].iloc[0]
elected

candidate_id                                                     2454
candidate_name                                             Naomi Long
candidate_party_name    Alliance - Alliance Party of Northern Ireland
candidate_party_id                                              PP103
constituency                                             belfast-east
count_stage                                                         1
count                                                          7610.0
quota                                                            6727
Name: 7, dtype: object

In [81]:
#Then need to go on to the next stage
transferable_votes = elected['count'] - quota
origin_party = elected['candidate_party_id']
transfers = df[df['count_stage']==2]
transfers

,candidate_id,candidate_name,candidate_party_name,candidate_party_id,constituency,count_stage,count,quota
14,8116,Andy Allen,Ulster Unionist Party,PP83,belfast-east,2,5313.17,6727
15,19719,Sheila Bodel,Conservative and Unionist Party,PP51,belfast-east,2,276.32,6727
16,13032,Joanne Bunting,Democratic Unionist Party - D.U.P.,PP70,belfast-east,2,6016.02,6727
17,19687,David Douglas,Democratic Unionist Party - D.U.P.,PP70,belfast-east,2,4435.18,6727
18,19745,Séamas de Faoite,SDLP (Social Democratic & Labour Party),PP55,belfast-east,2,260.45,6727
19,13035,Andrew Girvin,Traditional Unionist Voice - TUV,PP680,belfast-east,2,918.65,6727
20,13037,John Kyle,Progressive Unionist Party of Northern Ireland,PP101,belfast-east,2,2673.29,6727
21,2454,Naomi Long,Alliance - Alliance Party of Northern Ireland,PP103,belfast-east,2,6727.00,6727
22,13038,Chris Lyttle,Alliance - Alliance Party of Northern Ireland,PP103,belfast-east,2,5760.03,6727
23,19746,Jordy McKeag,Independent,ynmp-party:2,belfast-east,2,85.43,6727


In [84]:
transfers = df.set_index(['candidate_party_id','candidate_id','count_stage']).sort_index()['count'].groupby('candidate_id').diff()

In [85]:
party_transfers = transfers.unstack('count_stage')[2].groupby('candidate_party_id').sum()
party_transfers[origin_party]+=transferable_votes

In [88]:
(party_transfers/quota).sort_values()

candidate_party_id
PP51            0.000196
ynmp-party:2    0.000213
PP680           0.000245
PP4037          0.001112
PP39            0.001537
PP55            0.001553
PP101           0.002273
PP70            0.002780
PP305           0.004023
PP83            0.005674
PP103           0.104211
Name: 2, dtype: float64

In [160]:
transfers = df.set_index(
    ['candidate_party_name','candidate_id','count_stage']
).sort_index()['count'].groupby('candidate_id').diff().unstack('count_stage')
transfers[transfers>0].sum()

count_stage
1        0.00
2      832.92
3       83.21
4      246.01
5      710.80
6      894.65
7      622.02
8     1463.98
9     2466.06
10     163.00
11     525.98
dtype: float64

In [161]:
elected = []
excluded = []
transferables = []
seats = 5
for stage, sdf in df.groupby('count_stage'):
    transferable_votes = 0
    candidates = sdf[
            (~sdf['candidate_id'].isin({e['candidate_id'] for e in elected})) &\
            (~sdf['candidate_id'].isin({e['candidate_id'] for e in excluded}))
        ]
    quota_candidates = candidates[(sdf['count']>quota)].sort_values(ascending=False,by='count')
    if quota_candidates.size:
        # add top to elected, do transfers next round
        for _,candidate in quota_candidates.iterrows():
            candidate['elected_in']=stage
            elected.append(candidate)
            transferable_votes += candidate['count'] - quota
            print(f"{candidate['candidate_name']} elected in {stage} with {transferable_votes} to transfer")
    else:
        excluded_candidates = candidates[candidates['count']==0]
        for _,candidate in excluded_candidates.iterrows():
            excluded_candidate = candidate.to_dict()
            excluded_candidate['excluded_in']=stage
            excluded.append(excluded_candidate)
            transferable_votes += candidate['count']
            print(f"{excluded_candidate['candidate_name']} excluded in {stage} with {candidate['count']} to transfer")
    
    transferables.append(transferable_votes)
        
remainder = seats - len(elected)
if remainder:
    print(f"{remainder} seats remain")

Naomi Long elected in 1 with 883.0 to transfer


C:\Users\me\AppData\Local\Temp\ipykernel_29884\2080685566.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  quota_candidates = candidates[(sdf['count']>quota)].sort_values(ascending=False,by='count')


Jordy McKeag excluded in 3 with 0.0 to transfer
Séamas de Faoite excluded in 4 with 0.0 to transfer
Sheila Bodel excluded in 5 with 0.0 to transfer
Courtney Robinson excluded in 5 with 0.0 to transfer
Andrew Girvin excluded in 6 with 0.0 to transfer
Mairéad O'Donnell excluded in 7 with 0.0 to transfer
Chris Lyttle elected in 8 with 541.8699999999999 to transfer


C:\Users\me\AppData\Local\Temp\ipykernel_29884\2080685566.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  quota_candidates = candidates[(sdf['count']>quota)].sort_values(ascending=False,by='count')


Andy Allen elected in 9 with 530.6199999999999 to transfer
Joanne Bunting elected in 9 with 562.6300000000001 to transfer


C:\Users\me\AppData\Local\Temp\ipykernel_29884\2080685566.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  quota_candidates = candidates[(sdf['count']>quota)].sort_values(ascending=False,by='count')


John Kyle excluded in 10 with 0.0 to transfer
Georgina Milne excluded in 10 with 0.0 to transfer
1 seats remain


In [133]:
sdf[~sdf['candidate_id'].isin({e['candidate_id'] for e in elected})].sort_values(by='count', ascending=False)

,candidate_id,candidate_name,candidate_party_name,candidate_party_id,constituency,count_stage,count,quota
151,13040,Robin Newton,Democratic Unionist Party - D.U.P.,PP70,belfast-east,11,5541.65,6727
143,19687,David Douglas,Democratic Unionist Party - D.U.P.,PP70,belfast-east,11,5410.84,6727
141,19719,Sheila Bodel,Conservative and Unionist Party,PP51,belfast-east,11,0.00,6727
144,19745,Séamas de Faoite,SDLP (Social Democratic & Labour Party),PP55,belfast-east,11,0.00,6727
145,13035,Andrew Girvin,Traditional Unionist Voice - TUV,PP680,belfast-east,11,0.00,6727
146,13037,John Kyle,Progressive Unionist Party of Northern Ireland,PP101,belfast-east,11,0.00,6727
149,19746,Jordy McKeag,Independent,ynmp-party:2,belfast-east,11,0.00,6727
150,19699,Georgina Milne,Green Party,PP305,belfast-east,11,0.00,6727
152,19680,Mairéad O'Donnell,Sinn Féin,PP39,belfast-east,11,0.00,6727
153,13041,Courtney Robinson,Cross-Community Labour Alternative,PP4037,belfast-east,11,0.00,6727


In [134]:
sdf[sdf['candidate_id'].isin({e['candidate_id'] for e in elected})].sort_values(by='count', ascending=False)

,candidate_id,candidate_name,candidate_party_name,candidate_party_id,constituency,count_stage,count,quota
142,13032,Joanne Bunting,Democratic Unionist Party - D.U.P.,PP70,belfast-east,11,6759.01,6727
140,8116,Andy Allen,Ulster Unionist Party,PP83,belfast-east,11,6727.00,6727
147,2454,Naomi Long,Alliance - Alliance Party of Northern Ireland,PP103,belfast-east,11,6727.00,6727
148,13038,Chris Lyttle,Alliance - Alliance Party of Northern Ireland,PP103,belfast-east,11,6727.00,6727


### Need to think about this differently; 

The 'count stage' actually has two passes that are compressed into one action if you go purely by the results/outcomes. 

* Assess the current vote tally, including any transfers from the previous round
* Flag one of more individuals as elected/excluded, their transfers to be 'executed' in the next round. 

So in 'input' to the stages isn't the final counts from the last round but is the 

In [314]:
from collections import defaultdict

quota = df['quota'].max()

elected = []
excluded = []
transferables = defaultdict(list)

seats = 5
candidate_final_tallies = df.groupby('candidate_id')['count'].max()

for stage, sdf in df.groupby('count_stage'):
    
    candidates = sdf[
            (~sdf['candidate_id'].isin({e['candidate_id'] for e in elected})) &\
            (~sdf['candidate_id'].isin({e['candidate_id'] for e in excluded}))
        ].sort_values(ascending=True, by='count')
    
    remainder = seats - len(elected)
    
    if remainder == len(candidates.index): # Stages have ended, take whomever is left:
        for _, candidate in candidates.iterrows():
            elected_candidate = candidate.to_dict()
            elected_candidate['elected_in']=stage
            elected.append(elected_candidate)
            print(f"{elected_candidate['candidate_name']} elected in {stage}")
        break
        
    if stage > 1:
        # This is black magic taken from the Electoral Reform Society; 
        # 5.2.5 If, after all surpluses have been transferred or deferred, one or more places remain to be filled, the candidate or candidates with the fewest votes must be excluded. Exclude as many candidates together as possible, provided that:
        # (a) Sufficient candidates remain to fill all the remaining places
        # (b) The total votes of these candidates, together with the total of any deferred surpluses, does not exceed the vote of the candidate next above.

        candidates['cumsum'] = candidates['count'].cumsum()
        candidates['excluded'] = candidates['count'].shift(-1)>candidates['cumsum']
        for _,candidate in candidates[candidates['excluded']].iterrows():
            excluded_candidate = candidate.to_dict()
            if excluded_candidate['count'] <= 0.0:

                excluded_candidate['count'] = candidate_final_tallies[excluded_candidate['candidate_id']]
            excluded_candidate['excluded_in']=stage
            excluded.append(excluded_candidate)
            transferable_votes = excluded_candidate['count']
            transferables[stage+1].append((excluded_candidate, transferable_votes))
            print(f"{excluded_candidate['candidate_name']} excluded in {stage} with {transferable_votes} to transfer")
        

    # add top to elected, do transfers next round
    for _,candidate in candidates[candidates['count']>quota][::-1].iterrows():
        elected_candidate = candidate.to_dict()
        elected_candidate['elected_in']=stage
        elected.append(elected_candidate)
        transferable_votes = elected_candidate['count'] - quota
        transferables[stage+1].append((elected_candidate, transferable_votes))
        print(f"{elected_candidate['candidate_name']} elected in {stage} with {transferable_votes} to transfer")
    

    

Naomi Long elected in 1 with 883.0 to transfer
Jordy McKeag excluded in 2 with 85.43 to transfer
Séamas de Faoite excluded in 3 with 260.56 to transfer
Sheila Bodel excluded in 4 with 280.43 to transfer
Courtney Robinson excluded in 4 with 467.81 to transfer
Andrew Girvin excluded in 5 with 958.87 to transfer
Mairéad O'Donnell excluded in 6 with 1235.1 to transfer
Georgina Milne excluded in 7 with 2080.65 to transfer
John Kyle excluded in 8 with 3148.79 to transfer
Chris Lyttle elected in 8 with 541.8699999999999 to transfer
David Douglas excluded in 9 with 4995.25 to transfer
Andy Allen elected in 9 with 530.6199999999999 to transfer
Joanne Bunting elected in 9 with 32.01000000000022 to transfer
Robin Newton elected in 10


In [296]:
{e['candidate_name'] for e in elected}

{'Andy Allen', 'Chris Lyttle', 'Joanne Bunting', 'Naomi Long', 'Robin Newton'}

In [306]:
transfers

count_stage                                                  1       2   \
candidate_party_name                           candidate_id               
Alliance - Alliance Party of Northern Ireland  2454         NaN -883.00   
                                               13038        NaN  701.03   
Conservative and Unionist Party                19719        NaN    1.32   
Cross-Community Labour Alternative             13041        NaN    7.48   
Democratic Unionist Party - D.U.P.             13032        NaN    9.02   
                                               13040        NaN    5.50   
                                               19687        NaN    4.18   
Green Party                                    19699        NaN   27.06   
Independent                                    19746        NaN    1.43   
Progressive Unionist Party of Northern Ireland 13037        NaN   15.29   
SDLP (Social Democratic & Labour Party)        19745        NaN   10.45   
Sinn Féin                                      19680        NaN   10.34   
Traditional Unionist Voice - TUV               13035        NaN    1.65   
Ulster Unionist Party                          8116         NaN   38.17   

count_stage                                                     3       4   \
candidate_party_name                           candidate_id                  
Alliance - Alliance Party of Northern Ireland  2454           0.00    0.00   
                                               13038         20.22  105.61   
Conservative and Unionist Party                19719          2.11    2.00   
Cross-Community Labour Alternative             13041          7.00   11.33   
Democratic Unionist Party - D.U.P.             13032          5.00    5.11   
                                               13040          2.00    3.00   
                                               19687          1.00    4.00   
Green Party                                    19699         26.22   39.21   
Independent                                    19746        -85.43    0.00   
Progressive Unionist Party of Northern Ireland 13037          5.22    7.22   
SDLP (Social Democratic & Labour Party)        19745          0.11 -260.56   
Sinn Féin                                      19680          2.11   31.21   
Traditional Unionist Voice - TUV               13035          2.00    4.11   
Ulster Unionist Party                          8116          10.22   33.21   

count_stage                                                      5       6   \
candidate_party_name                           candidate_id                   
Alliance - Alliance Party of Northern Ireland  2454            0.00    0.00   
                                               13038         146.95   39.33   
Conservative and Unionist Party                19719        -280.43    0.00   
Cross-Community Labour Alternative             13041        -467.81    0.00   
Democratic Unionist Party - D.U.P.             13032          31.22  160.11   
                                               13040          35.00   88.11   
                                               19687          15.22  113.00   
Green Party                                    19699         223.87   38.22   
Independent                                    19746           0.00    0.00   
Progressive Unionist Party of Northern Ireland 13037          66.22  184.33   
SDLP (Social Democratic & Labour Party)        19745           0.00    0.00   
Sinn Féin                                      19680          16.22    2.22   
Traditional Unionist Voice - TUV               13035          34.11 -958.87   
Ulster Unionist Party                          8116          141.99  269.33   

count_stage                                                       7        8   \
candidate_party_name                           candidate_id                     
Alliance - Alliance Party of Northern Ireland  2454             0.00     0.00   
                                          

In [381]:
## Calculate Transfers 
transfers = df.set_index(
    ['candidate_party_id','candidate_id','count_stage']
).sort_index()['count'].groupby('candidate_id').diff().unstack('count_stage')


transfer_observations = defaultdict(list) # party -> list(dict)

for stage, transfer_instances in transferables.items():
    potential_transfers = sum((v for t,v in transfer_instances))
    print(stage, {(t['candidate_name'], v) for t,v in transfer_instances}, transfers[transfers>0][stage].sum()/potential_transfers)
    this_stage = transfers[transfers>0][stage]
    
    party_transfers = this_stage.reset_index('candidate_id', drop=True).groupby('candidate_party_id').sum()
    party_transfers['non_transferable'] = potential_transfers - party_transfers.sum()
    
    for t,v in transfer_instances:
        # Get the transfers and add them proportional to the ratio between the transfer instance vote and the total potential

        transfer_observations[t['candidate_party_id']].append(
           (party_transfers/party_transfers.sum()*(v/potential_transfers)).to_dict()
        )
        
        

2 {('Naomi Long', 883.0)} 0.9432842582106459
3 {('Jordy McKeag', 85.43)} 0.9740138124780632
4 {('Séamas de Faoite', 260.56)} 0.9441587350322354
5 {('Courtney Robinson', 467.81), ('Sheila Bodel', 280.43)} 0.9499625788517051
6 {('Andrew Girvin', 958.87)} 0.9330253319010913
7 {("Mairéad O'Donnell", 1235.1)} 0.5036191401505952
8 {('Georgina Milne', 2080.65)} 0.7036166582558335
9 {('John Kyle', 3148.79), ('Chris Lyttle', 541.8699999999999)} 0.6681894295329293
10 {('Joanne Bunting', 32.01000000000022), ('David Douglas', 4995.25), ('Andy Allen', 530.6199999999999)} 0.029327729278070053


In [391]:
transfer_observations

defaultdict(list,
            {'PP103': [{'PP101': 0.017315968289920682,
               'PP103': 0.7939184597961492,
               'PP305': 0.030645526613816473,
               'PP39': 0.011710079275198095,
               'PP4037': 0.008471121177802965,
               'PP51': 0.0014949037372593355,
               'PP55': 0.011834654586636453,
               'PP680': 0.0018686296715741532,
               'PP70': 0.021177802944508186,
               'PP83': 0.043227633069082755,
               'ynmp-party:2': 0.0016194790486976295,
               'non_transferable': 0.05671574178935413},
              {'PP101': 0.0,
               'PP103': 0.0,
               'PP305': 0.0,
               'PP39': 0.0,
               'PP4037': 0.0,
               'PP51': 0.0,
               'PP55': 0.0,
               'PP680': 0.0,
               'PP70': 0.05000840912281649,
               'PP83': 0.04809648435992325,
               'ynmp-party:2': 0.0,
               'non_transferable': 0.048717084158115

In [370]:
this_stage.sum()

832.9200000000003

In [317]:
# observed transfers
transfers[transfers>0].sum()

count_stage
1        0.00
2      832.92
3       83.21
4      246.01
5      710.80
6      894.65
7      622.02
8     1463.98
9     2466.06
10     163.00
11     525.98
dtype: float64